# Réseau de Neurones et Dropout

## I. Classification  avec les données MNIST

Nous nous appuyons dans un premier temps sur la base "MNIST" pour mettre en oeuvre l'utilisation du dropout dans le cas de la classification. Cette base est elle-même utilisée par les auteurs dans leur article. Cette base est constituée d'images représentant les chiffres de 1 à 10. L'objectif est de classifier les différentes images selon leur appartenance à ces dix chiffres. Pour des raisons de mémoire et pour simplifier la tâche ici, nous créons une sous base comprenant seulement les chiffres 0 et 1 en proportions égales. Nous constituons une base de test constituée de 1000 images et une base d'apprentissage composée de 2000 images. Nous définissons un réseau de neurones à deux couches cachées. 

### Les données

In [1]:
# Téléchargement des données
from sklearn.datasets import fetch_mldata
custom_data_home = "/cours/statistique_bayesienne/data"
mnist = fetch_mldata('MNIST original', data_home=custom_data_home)

In [2]:
# Téléchargement des packages
import numpy as np
import sklearn.datasets
import pandas as pd

In [3]:
# séparation des données
X = mnist.data
Y = mnist.target
random_x=[i for i in range(1000)]

In [4]:
# base d'apprentissage/base de test
X_train = X[:1000, ]
Y_train = Y[:1000, ]
X_train_bis = X[7000:8000, ]
Y_train_bis = Y[7000:8000, ]
X_test = X[5500:6500, ]
Y_test = Y[5500:6500, ]
X_tr = np.concatenate((X_train, X_train_bis))
Y_tr = np.concatenate((Y_train, Y_train_bis))
X_tr = X_tr.astype(int)
Y_tr = Y_tr.astype(int)

In [5]:
from collections import Counter
Counter(Y_tr)
Counter(Y_test)

Counter({0.0: 423, 1.0: 577})

In [6]:
# Définition de la probabilité de dropout
dropout_percent = 0.5

### Implémentation du réseau de neurones FNN deux couches cachées

In [7]:
# Définition de la fonction de coût, il s'agit de la fonction à minimser avec la méthode de descente de gradient
def calculate_loss(model, X, y):
    w1, b1, w2, b2, w3, b3 = model['w1'], model['b1'], model['w2'], model['b2'], model['w3'], model['b3']
    # Forward propagation to calculate our predictions
    z1 = X.dot(w1) + b1
    l1 = np.tanh(z1)
    z2 = l1.dot(w2) + b2
    l2 = np.tanh(z2)
    z3 = l2.dot(w3) + b3
    exp_scores = np.exp(z3)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    corect_logprobs = -np.log(probs[range(num_x), y])
    data_loss = np.sum(corect_logprobs)
    data_loss += reg_lambda / 6 * (np.sum(np.square(w1)) + np.sum(np.square(w2)) + np.sum(np.square(w3)) + np.sum(np.square(b1))
                                   + np.sum(np.square(b2)) + np.sum(np.square(b3)))

    return 1. / num_x * data_loss

In [8]:
# Fonction construisant le modèle
def build_model(hdim_1, hdim_2, num_it, X, y):

    # Initialisation aléatoire des paramètres
    np.random.seed(0)
    w1 = np.random.randn(input_dim, hdim_1) / np.sqrt(input_dim)
    b1 = np.zeros((1, hdim_1))
    w2 = np.random.randn(hdim_1, hdim_2) / np.sqrt(hdim_1)
    b2 = np.zeros((1, hdim_2))
    w3 = np.random.randn(hdim_2, output_dim) / np.sqrt(hdim_2)
    b3 = np.zeros((1, output_dim))

    model = {}

    for i in range(0, num_it):

        # Forward propagation
        z1 = X.dot(w1) + b1
        l1 = np.tanh(z1)
        z2 = l1.dot(w2) + b2
        l2 = np.tanh(z1)
        l2 *= np.random.binomial([np.ones((len(X), hdim_2))],
                                 1 - dropout_percent)[0] * (1.0 / (1 - dropout_percent))
        z3 = l2.dot(w3) + b3
        exp_scores = np.exp(z3)
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

        # Backpropagation
        delta4 = probs
        delta4[range(num_x), y] -= 1
        dw3 = (l2.T).dot(delta4)
        db3 = np.sum(delta4, axis=0, keepdims=True)
        delta3 = delta4.dot(w3.T) * (1 - np.power(l2, 2))
        dw2 = (l1.T).dot(delta3)
        db2 = np.sum(delta3, axis=0, keepdims=True)
        delta2 = delta3.dot(w2.T) * (1 - np.power(l1, 2))
        dw1 = np.dot(X.T, delta2)
        db1 = np.sum(delta2, axis=0)

        # Terme de régularisation
        dw2 += reg_lambda / 3 * w2
        dw1 += reg_lambda / 3 * w1
        dw3 += reg_lambda / 3 * w3
        db1 += reg_lambda / 3 * b1[:, 0]
        db2 += reg_lambda / 3 * b2
        db3 += reg_lambda / 3 * b3

        #  Mise à jour des paramètres dans la descente de gradient
        w1 += -epsilon * dw1
        b1 += -epsilon * db1
        w2 += -epsilon * dw2
        b2 += -epsilon * db2
        w3 += -epsilon * dw3
        b3 += -epsilon * db3

        model = {'w1': w1, 'b1': b1, 'w2': w2, 'b2': b2, 'w3': w3, 'b3': b3}

        # Calcul de l'erreur
        if i % 1000 == 0:
            print("Loss after iteration %i: %f" %
                  (i, calculate_loss(model, X, y)))

    return model

In [9]:
# Apprentissage du modèle pour des paramètres données, deux couches cachées de 20 neurones, 10000 itérations
num_x = len(X_tr)
input_dim = 784
output_dim = 2
epsilon = 0.001
reg_lambda = 0.005
model = build_model(20, 20, 10000, X_tr, Y_tr)

Loss after iteration 0: 0.425489
Loss after iteration 1000: 0.059366
Loss after iteration 2000: 0.134281
Loss after iteration 3000: 0.308279
Loss after iteration 4000: 0.574985
Loss after iteration 5000: 0.948007
Loss after iteration 6000: 1.415561
Loss after iteration 7000: 1.980560
Loss after iteration 8000: 2.644123
Loss after iteration 9000: 3.413304


### Base de test et dropout

On effectue alors le dropout 100 fois en utilisant une loi binomiale appliquée aux poids de la dernière couche cachée. La loi utilisée pour effectuée le dropout est la loi de Bernoulli.

In [10]:
# Fonction lançant le dropout à utiliser sur les observations de la base de test
def dropout(model, x, dropout_percent):
    w1, b1, w2, b2, w3, b3 = model['w1'], model['b1'], model['w2'], model['b2'], model['w3'], model['b3']
    # Forward propagation
    z1 = x.dot(w1) + b1
    l1 = np.tanh(z1)
    z2 = l1.dot(w2) + b2
    l2 = np.tanh(z1)
    # Mise en place du dropout avec binomiale
    l2 *= np.random.binomial([np.ones((len(x), hdim_1))],
                             1 - dropout_percent)[0] * (1.0 / (1 - dropout_percent))
    z3 = l2.dot(w3) + b3
    exp_scores = np.exp(z3)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    #"the model uncertainty in the softmax output can be summarised by taking the mean of the distribution"
    return probs

On teste avec une probabilité de dropout de 0.02. On effectue 100 dropout aléatoires sur le modèle pour obtenir l'incertitude du modèle.

In [11]:
# Prédictions des étiquettes (1 ou 0) pour les 1000 observations de la base de test en utilisant la méthode dropout
# predictive_mean contient les probabilités finales, et predictive_variance contient la variance de la loi a posteriori
hdim_1 = 20
hdim_2 = 20
probs_tot = []
T = 100
dropout_percent = 0.5
l = 2
for _ in range(T):
    probs = dropout(model, X_test, dropout_percent)
    probs_tot += [probs.T[1]]

    predictive_mean = np.mean(probs_tot, axis=0)
predictive_variance = np.var(probs_tot, axis=0)
tau = l**2 * (1 - dropout_percent) / (2 * len(X_test) * reg_lambda)
predictive_variance += tau**-1

In [12]:
# on transforme predictive_mean en valeurs d'étiquettes 0 ou 1
predictive_class = [1 if i >= 0.5 else 0 for i in predictive_mean]

# Section dédiée à la visualisation des résultats et de l'incertitude

In [13]:
# importation des packages nécessaires
from sklearn.metrics import confusion_matrix
import plotly
from plotly.graph_objs import Scatter, Layout
import plotly.graph_objs as go

In [14]:
# matrice de confusion des résultats
m = confusion_matrix(Y_test, predictive_class, labels=None, sample_weight=None)
m

array([[420,   3],
       [  0, 577]], dtype=int64)

Petite transformation décimale des résultats pour pouvoir les visualiser

In [15]:
u = np.array(probs_tot).T
for i in range(1000):
    for j in range(100):
        u[i, j] = int(u[i, j] * 1000) / 1000

u1 = np.concatenate((u, np.array(predictive_class, ndmin=2).T), axis=1)
u1 = u1[u1[:, 100].argsort()]
# u1 contient les probabilités prédites pour chaque boucle sur le dropout mais également les étiquettes prédites
# triées pour une meilleure visualisation

In [16]:
# graphiques permettant de visualiser l'incertitude sur les résultats (seulement 50 prédictions affichées par commodité)

data = []

for i in range(50):
    trace0 = go.Scatter(
        x=random_x,
        y=u1[:, i],
        mode='markers',
        name='markers',
        marker=dict(
            size=2,
            color='rgba(255, 182, 193, .9)',
            line=dict(
                width=1,
            )
        ))

    data.append(trace0)

trace1 = go.Scatter(
    x=random_x,
    y=u1[:, 100],
    mode='lines',
    name='lines'
)
data.append(trace1)


plotly.offline.plot(data)

'file://C:\\Users\\asus\\Documents\\ENSAE\\3A\\3A ENSAE\\Stat Bay\\Code_readme_Bresson_Iakovlev_Roblin (2)\\temp-plot.html'

# Partie Modèle Dropout régresseur, nouvelle données à charger

Les données étudiées ici sont des séries temporelles portant sur la consommation totale en électricité d'un foyer sur 5 ans. Nous cherchons à prédire la consommation en temps réel à l'aide de 5 variables explicatives représentant des mesures de tension, intensité etc... dans le foyer. Nous chercherons à visualiser l'incertitude de prédiction à l'aide d'un modèle Dropout.

In [17]:
# chargement des données
base = pd.read_csv('C:/Users/asus/Documents/ENSAE/3A/3A ENSAE/Stat Bay/household_power_consumption.txt', sep=';')

C:\Users\asus\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning:

Columns (2,3,4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.



In [18]:
# on enlève les lignes avec des nan
base = base.dropna(axis=0, how='any')

In [19]:
# séparation base de test et base d'apprentissage

Xreg_train = np.array(base)[:50000, 3:]
Yreg_train = np.array(base)[:50000, 2]
Xreg_test = np.array(base)[50001:75000, 3:]
Yreg_test = np.array(base)[50001:75000, 2]
Xreg_tr = Xreg_train.astype(float)
Yreg_tr = np.array(Yreg_train.astype(float), ndmin=2).T

Xreg_tt = Xreg_test.astype(float)
Yreg_tt = Yreg_test.astype(float)

In [20]:
# Création de la fonction d'activation Relu et de sa dérivée
def ReLU(x):
    return x * (x > 0)


def dReLU(x):
    if x > 0:
        return 1
    else:
        return 0

In [21]:
# fonction de perte pour la régression
def calculate_loss(model, X, y):
    w1, b1, w3, b3 = model['w1'], model['b1'], model['w3'], model['b3']
    # Forward propagation to calculate our predictions
    z1 = X.dot(w1) + b1
    l1 = np.array([ReLU(x) for x in z1])
    z3 = l1.dot(w3) + b3
    prediction = np.array([ReLU(x) for x in z3])
    loss = 1 / len(prediction) * np.sum(np.square(prediction - y)) + reg_lambda / 4 * ((np.sum(np.square(w1)) + np.sum(np.square(w3))) +
                                                                                       (np.sum(np.square(b1)) + +np.sum(np.square(b3))))
    return loss

In [22]:
# construction du modèle
def build_model(hdim_1, num_it, X, y, dropout_percent):

    # Initialisation des paramètres
    np.random.seed(0)

    w1 = np.random.randn(input_dim, hdim_1) / np.sqrt(input_dim)
    b1 = np.zeros((X.shape[0], hdim_1))
    w3 = np.random.randn(hdim_1, output_dim) / np.sqrt(hdim_1)
    b3 = np.zeros((X.shape[0], output_dim))

    model = {}

    for i in range(0, num_it):

        # Forward propagation
        z1 = X.dot(w1) + b1
        l1 = np.array([ReLU(x) for x in z1])
        drops = np.random.binomial(
            [np.ones((len(X), hdim_1))], 1 - dropout_percent)[0] * (1.0 / (1 - dropout_percent))
        l1 *= drops
        z3 = l1.dot(w3) + b3
        prediction = z3

        # Backprop to compute gradients of w1 and w3 with respect to loss
        grad_y_pred = 2 * (prediction - y)  # the last layer's error
        dw3 = l1.T.dot(grad_y_pred)
        db3 = grad_y_pred
        dw3 = dw3 / np.linalg.norm(dw3, ord=None)
        grad_h_relu2 = grad_y_pred.dot(w3.T)  # the second laye's error
        grad_h2 = grad_h_relu2.copy()
        grad_h2[z1 < 0] = 0  # the derivate of ReLU
        sigmaprim = np.array([[dReLU(z1[i, j])
                               for j in range(20)]for i in range(50000)])
        dw1 = X.T.dot(grad_h2)
        dw1 = dw1 / np.linalg.norm(dw1, ord=None)
        db1 = grad_h2

        # Terme de régularisation

        dw1 += reg_lambda / 4 * w1
        dw3 += reg_lambda / 4 * w3
        db1 += reg_lambda / 4 * b1
        db3 += reg_lambda / 4 * b3

        #  Mise à jour des paramètres dans la descente de gradient
        w1 += -epsilon * dw1
        b1 += -epsilon * db1
        w3 += -epsilon * dw3
        b3 += -epsilon * db3

        model = {'w1': w1, 'b1': b1, 'w3': w3, 'b3': b3}

        # Calcul de l'erreur
        if i % 100 == 0:
            print("Loss after iteration %i: %f" %
                  (i, calculate_loss(model, X, y)))

    return model

In [23]:
# implémentation du modèle
num_x = len(Xreg_tr)
input_dim = 6
output_dim = 1
epsilon = 0.001
reg_lambda = 0.005
dropout_percent = 0.05
model = build_model(20, 1000, Xreg_tr, Yreg_tr, 0.05)

Loss after iteration 0: 47.643259
Loss after iteration 100: 22.421216
Loss after iteration 200: 34.668802
Loss after iteration 300: 33.411170
Loss after iteration 400: 28.894118
Loss after iteration 500: 26.419170
Loss after iteration 600: 24.800779
Loss after iteration 700: 23.869259
Loss after iteration 800: 23.545728
Loss after iteration 900: 23.665117


In [24]:
# définition de la fonction dropout
def dropout(model, X, dropout_percent):
    w1, b1, w3, b3 = model['w1'], model['b1'], model['w3'], model['b3']
    # Forward propagation
    z1 = X.dot(w1) + b1[0:X.shape[0], ]
    l1 = np.array([ReLU(x) for x in z1])
    drops = np.random.binomial([np.ones((len(X), hdim_1))],
                               1 - dropout_percent)[0] * (1.0 / (1 - dropout_percent))
    l1 *= drops
    z3 = l1.dot(w3) + b3[0:X.shape[0], ]
    prediction = z3
    #"the model uncertainty in the softmax output can be summarised by taking the mean of the distribution"
    return prediction

In [25]:
# méthode dropout appliquée à la base de test sur 100 itérations
hdim_1 = 20
prediction_tot = []
T = 100
dropout_percent = 0.05
l = 2
for _ in range(T):
    prediction = dropout(model, Xreg_tt, dropout_percent)
    prediction_tot += [prediction.T]
predictive_mean = np.mean(prediction_tot, axis=0)
predictive_variance = np.var(prediction_tot, axis=0)
tau = l**2 * (1 - dropout_percent) / (2 * len(Xreg_tt) * reg_lambda)
predictive_variance += tau**-1

# Section dédiée à la visualisation

In [26]:
# transformation des données pour visualisation
u = np.array(prediction_tot).T[:, 0]

In [27]:
# Premier graphique représentant la courbes des valeurs réelles et la courbes des valeurs prédites
random_x = [i for i in range(1000)]

# Create traces
trace0 = go.Scatter(
    x=random_x,
    y=predictive_mean.T[0:1000, 0],
    mode='lines',
    name='Predictions',
    marker=dict(
        size=2,
        color='blue',
        line=dict(
            width=1,
        )
    ))
trace1 = go.Scatter(
    x=random_x,
    y=Yreg_tt[0:1000],
    mode='lines',
    name='True values',
    marker=dict(
        size=2,
        color='grey',
        line=dict(
            width=1,
        )
    )
)


data = [trace0, trace1]
plotly.offline.plot(data)

'file://C:\\Users\\asus\\Documents\\ENSAE\\3A\\3A ENSAE\\Stat Bay\\Code_readme_Bresson_Iakovlev_Roblin (2)\\temp-plot.html'

In [28]:
# Second graphique représentant l'incertitude
data = []

for i in range(50):
    trace0 = go.Scatter(
        x=random_x,
        y=u[0:1000, i],
        mode='markers',
        name='markers',
        marker=dict(
            size=2,
            color='rgba(255, 182, 193, .9)',
            line=dict(
                width=1,
            )
        ))

    data.append(trace0)

trace1 = go.Scatter(
    x=random_x,
    y=predictive_mean.T[0:1000, 0],
    mode='lines',
    name='lines'
)
data.append(trace1)


plotly.offline.plot(data)

'file://C:\\Users\\asus\\Documents\\ENSAE\\3A\\3A ENSAE\\Stat Bay\\Code_readme_Bresson_Iakovlev_Roblin (2)\\temp-plot.html'